In [1]:
import numpy as np
from numpy import exp,array,random,dot
import pandas as pd

In [2]:
def normalize(matrix):
    n = matrix.shape[1]
    norm_matrix = matrix

    for i in range(n):
        col_mean = np.mean(matrix[:,i])
        col_stdv = np.std(matrix[:,i])
    norm_matrix[:,i] = (norm_matrix[:,i] - col_mean) / col_stdv
    return norm_matrix

def normalize1(matrix):
    return (matrix - matrix.mean()) / matrix.std()

In [3]:
# Reading data from csv
data = pd.read_csv("data/train.csv")

In [4]:
# Classifier data entries to dummy coloumns
#data = pd.get_dummies(data)
#data.to_csv("data/data_afterdum.csv", sep=',',header=True,index=False)

In [5]:
# Spliting the data into Training data and Validation data
train, validate = np.split(data.sample(frac=1), [int(.7*len(data))])
print(len(train),len(validate))
#print(train)
#print(validate)

27281 11692


In [6]:
trn_ids = train[['id']]
#print(train_ids)
trn_y = train[['salary']]
#print(train_y)
trn_features = train.drop('id', axis=1)
trn_features = trn_features.drop('salary', axis=1)
trn_features = trn_features.drop('race', axis=1)
trn_features = trn_features.drop('native-country', axis=1)
trn_features = trn_features.drop('education', axis=1)#correlated
trn_features = pd.get_dummies(trn_features)
trn_features = normalize1(trn_features)
print(trn_features.shape)

(27281, 45)


In [7]:
vld_ids = validate[['id']]
#print(valid_ids)
vld_y = validate[['salary']]
#print(train_y)
vld_features = validate.drop('id', axis=1)
vld_features = vld_features.drop('salary', axis=1)
vld_features = vld_features.drop('native-country', axis=1)
vld_features = vld_features.drop('race', axis=1)
vld_features = vld_features.drop('education', axis=1)
vld_features = pd.get_dummies(vld_features)
vld_features = normalize1(vld_features)
print(vld_features.shape)

(11692, 45)


In [8]:
test_data = pd.read_csv("data/test.csv")
test_ids = test_data['id']
#print(test_ids)
tst_features = test_data.drop('id', axis=1)
tst_features = tst_features.drop('native-country', axis=1)
tst_features = tst_features.drop('race', axis=1)
tst_features = tst_features.drop('education', axis=1)
tst_features = pd.get_dummies(tst_features)
tst_features = normalize1(tst_features)
print(tst_features.shape)

(6878, 45)


In [9]:
# Now Neural Network implementation starts

# Activation functions
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x))

def derv_sigmoid(x):
    return x * (1 - x)

def tanh(x):
        return np.tanh(x)

def derv_tanh(x):
        return (1.0 - np.square(np.tanh(x)))


In [12]:
class NeuralNetwork():
    def __init__(self):
        
        # Number of nodes in each Layer
        l1_nodes=45
        l2_nodes=30
        l3_nodes=30
        l4_nodes=1

        # Assigning random weights to Neural Network
        self.weights1 = 2 * random.random((l1_nodes, l2_nodes)) -1
        self.weights2 = 2 * random.random((l2_nodes, l3_nodes)) -1
        self.weights3 = 2 * random.random((l3_nodes, l4_nodes)) -1

    # Activation functions
    def tanh(self,x):
        return np.tanh(x)

    def derv_tanh(self,x):
        return (1.0 - (np.tanh(x))**2)

    # Train neural network using backpropogation
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations,eta):
        for i in range(number_of_training_iterations):
            # print(i)
            
            # Forward Pass
            a2 = self.tanh(dot(training_set_inputs, self.weights1))
            a3 = self.tanh(dot(a2, self.weights2))
            output = self.tanh(dot(a3, self.weights3))

            # Backpopogation Pass
            # Error for each layer
            del4 = (training_set_outputs - output)*self.derv_tanh(output)
            print(i,np.mean(np.square(del4)))
            del3 = dot(self.weights3, del4.T)*(self.derv_tanh(a3).T)
            del2 = dot(self.weights2, del3)*(self.derv_tanh(a2).T)

            # Adjustments (gradients) for each layer
            adjustment3 = eta*dot(a3.T, del4)
            adjustment2 = eta*dot(a2.T, del3.T)
            adjustment1 = eta*dot(training_set_inputs.T, del2.T)

            # Adjusting weights accordingly
            self.weights1 += adjustment1
            self.weights2 += adjustment2
            self.weights3 += adjustment3

        return self.weights1,self.weights2,self.weights3


In [14]:
# Initialising Neural Network
neural_network = NeuralNetwork()

#print ("Starting weights Layer 1: ")
#print (neural_network.weights1)
#print ("\nStarting weights Layer 2: ")
#print (neural_network.weights2)
#print ("\nStarting weights Layer 3: ")
#print (neural_network.weights3)

# Training Set
training_set_inputs = np.array(trn_features)
training_set_outputs = np.array(trn_y.T)

fweights1,fweights2,fweights3=neural_network.train(training_set_inputs, training_set_outputs.T,100,0.00003)

#print ("\nNew weights Layer 1 after training:")
#print (neural_network.weights1)
#print ("\nNew weights Layer 2 after training: ")
#print (neural_network.weights2)
#print ("\nNew weights Layer 3 after training: ")
#print (neural_network.weights3)

# Validation
after_l1 = tanh(np.dot(vld_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
after_l3 = tanh(np.dot(after_l2,fweights3))
#print("after_l3")
#print(after_l3)
predicted = []

after_l3 = after_l3.ravel()
#print(after_l3)

#Threshholding values
for i in range(len(after_l3)):
    if after_l3[i] > 0:
        predicted.append([1])
    else:
        predicted.append([0])

#Validating accuracy of Neural network results
vld_y = np.array(vld_y).ravel()
#print(vld_y)
#np.savetxt("data/vld_y.csv",vld_y,delimiter=',',fmt="%d",header="id,salary",comments ='')
c=0
for i in range(len(predicted)):
    if(predicted[i] == vld_y[i]):
        c+=1
vld_acur=c/len(predicted)
print(vld_acur)


0 0.248649201602
1 0.189103787201
2 0.210361197383
3 0.186884772336
4 0.204432964145
5 0.184958055687
6 0.197448300748
7 0.181145229779
8 0.189911967933
9 0.176294038928
10 0.182998397402
11 0.171484638459
12 0.177066321584
13 0.166837969761
14 0.171904799155
15 0.162365446981
16 0.167447792696
17 0.158194562286
18 0.163765899703
19 0.154527898058
20 0.161025091571
21 0.151670598221
22 0.159457032367
23 0.150055690556
24 0.159202132197
25 0.150031213935
26 0.15987657092
27 0.151225261618
28 0.160267864947
29 0.152280831058
30 0.159154945873
31 0.152059825707
32 0.156458684435
33 0.15065237888
34 0.152938266223
35 0.148746659876
36 0.149316821501
37 0.146888994593
38 0.145984443233
39 0.145368301226
40 0.143086531806
41 0.144325507401
42 0.140657287887
43 0.143844046577
44 0.138704985507
45 0.143980797918
46 0.137246402547
47 0.144725489065
48 0.136280411258
49 0.145873868225
50 0.135686528535
51 0.146890638844
52 0.135129819546
53 0.147032404732
54 0.134191164943
55 0.145856313824
56 0

In [ ]:
# Getting final results from test data
after_l1 = tanh(np.dot(tst_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
after_l3 = tanh(np.dot(after_l2,fweights3))
print("after_l3")
print(after_l3)
predicted = []
after_l3 = after_l3.ravel()

result=np.column_stack((test_ids,after_l3))
np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')
print(after_l3)
for i in range(len(after_l3)):
    if after_l3[i] > 0:
        predicted.append(1)
    else:
        predicted.append(0)
print("predicted")
print(predicted)
result=np.column_stack((test_ids,(after_l3))

np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')